In [1]:
import os

In [2]:
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction\\notebooks'

In [3]:
os.chdir('../')
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformed_filename: Path

In [6]:
from src.constants import *
from src.utils.helper import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH, schema_filepath = SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            transformed_filename = config.transformed_filename
        )

        return data_transformation_config


In [8]:
import os
from src import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def handle_missing_values(self):
        data = pd.read_csv(self.config.data_path)
        data.dropna(subset=['weather_code'], inplace=True)
        data.to_csv(os.path.join(self.config.root_dir, self.config.transformed_filename))

        logger.info("Missing value handled successfully")

    def feature_engineering(self):
        weather_codes = {
            0: "Clear sky",
            1: "Mainly clear",
            2: "Partly cloudy",
            3: "Overcast",
            45: "Fog",
            48: "Depositing rime fog",
            51: "Light drizzle",
            53: "Moderate drizzle",
            55: "Dense drizzle",
            56: "Light freezing drizzle",
            57: "Dense freezing drizzle",
            61: "Slight rain",
            63: "Moderate rain",
            65: "Heavy rain",
            66: "Light freezing rain",
            67: "Heavy freezing rain",
            71: "Slight snow fall",
            73: "Moderate snow fall",
            75: "Heavy snow fall",
            77: "Snow grains",
            80: "Slight rain showers",
            81: "Moderate rain showers",
            82: "Violent rain showers",
            85: "Slight snow showers",
            86: "Heavy snow showers",
            95: "Thunderstorm: slight or moderate",
            96: "Thunderstorm with slight hail",
            99: "Thunderstorm with heavy hail",
        }

        data = pd.read_csv(self.config.data_path)
        data['weather'] = data['weather_code'].apply(lambda x: weather_codes[x])
        data.drop(['weather_code', 'time'], axis=1, inplace=True)
        data.to_csv(os.path.join(self.config.root_dir, self.config.transformed_filename))

        logger.info('Feature engineering done successfully')

    def train_test_spliting(self):
        data = pd.read_csv(os.path.join(self.config.root_dir, self.config.transformed_filename))

        train, test = train_test_split(data, test_size=.25)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.handle_missing_values()
    data_transformation.feature_engineering()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-05-22 14:02:52,239: INFO: helper: yaml file: config\config.yaml loaded successfully]
[2025-05-22 14:02:52,323: INFO: helper: yaml file: params.yaml loaded successfully]
[2025-05-22 14:02:52,328: INFO: helper: yaml file: schema.yaml loaded successfully]
[2025-05-22 14:02:52,331: INFO: helper: created directory at: artifacts]
[2025-05-22 14:02:52,333: INFO: helper: created directory at: artifacts/data_transformation]
[2025-05-22 14:02:52,426: INFO: 1547636484: Missing value handled successfully]
[2025-05-22 14:02:52,489: INFO: 1547636484: Splited data into training and test sets]
[2025-05-22 14:02:52,491: INFO: 1547636484: Train shape: (65, 15)]
[2025-05-22 14:02:52,493: INFO: 1547636484: Test shape: (22, 15)]


## Data Transformation Testing

In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('artifacts/data_ingestion/weather_data.csv')

In [34]:
df.head()

,time,temperature_2m_max,temperature_2m_min,weather_code,apparent_temperature_max,apparent_temperature_min,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,uv_index_max,daylight_duration,sunshine_duration,et0_fao_evapotranspiration
0,2025-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44519.41,NaN,NaN
1,2025-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44494.91,NaN,NaN
2,2025-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44470.38,NaN,NaN
3,2025-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44445.79,NaN,NaN
4,2025-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44420.94,NaN,NaN


In [28]:
weather_codes = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    56: "Light freezing drizzle",
    57: "Dense freezing drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    66: "Light freezing rain",
    67: "Heavy freezing rain",
    71: "Slight snow fall",
    73: "Moderate snow fall",
    75: "Heavy snow fall",
    77: "Snow grains",
    80: "Slight rain showers",
    81: "Moderate rain showers",
    82: "Violent rain showers",
    85: "Slight snow showers",
    86: "Heavy snow showers",
    95: "Thunderstorm: slight or moderate",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail",
}


In [29]:
df['weather'] = df['weather_code'].apply(lambda x: weather_codes[x])

In [30]:
df.drop(['weather_code', 'time'], axis=1, inplace=True)

In [31]:
df.head()

,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,uv_index_max,daylight_duration,sunshine_duration,et0_fao_evapotranspiration,weather
0,29.1,20.1,34.4,23.0,10.0,13.0,31.0,16.23,8.75,44266.77,28800.00,3.29,Moderate rain showers
1,27.5,20.6,33.1,23.8,8.3,13.0,317.0,15.04,8.40,44240.46,22565.18,2.87,Moderate rain showers
2,26.3,20.6,32.5,24.1,9.4,18.4,207.0,18.17,8.70,44214.03,34493.40,3.47,Slight rain showers
3,26.8,20.7,34.9,23.9,5.7,17.3,60.0,13.68,8.65,44187.51,23368.11,2.77,Moderate rain showers
4,29.6,19.8,35.5,22.5,9.7,14.4,48.0,23.72,9.15,44160.92,36000.00,4.60,Slight rain showers
